# Session #8 Homework

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

## Model

In [2]:
# Create the convolutional base
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Add Dense layers on top
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(1))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

In [4]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
loss = tf.keras.losses.MeanSquaredError()
rmse = tf.keras.metrics.RootMeanSquaredError()

model.compile(optimizer=optimizer, loss=loss, metrics=[rmse])

In [5]:
# history = model.fit(train_images, train_labels, epochs=10, 
#                     validation_data=(test_images, test_labels))

## Question 1

Since we have a binary classification problem, what is the best loss function for us?

## Question 2

What's the total number of parameters of the model?

## Generators and Training

## Question 3

What is the median of training accuracy for this model?

## Question 4

What is the standard deviation of training loss for this model?

## Data Augmentation

## Question 5

What is the mean of validation loss for the model trained with augmentations?

## Question 6

What's the average of validation accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?